In [1]:
import pandas as pd

In [7]:
df = pd.read_csv(r'C:\Projects\36118 ANLP\Assignment 2\LLM\LegalEase_NSW\github_repo\eval\outputs\llm_generation_and_evaluation_results.csv')

In [8]:
# Remove API failures
df_success = df[df['generated_summary'] != 'API_CALL_FAILED']

# Model-level stats (averaged across ALL prompts)
model_stats = df_success.groupby('model_name').agg({
    'format_pass': 'mean',
    'bertscore_f1': 'mean',
    'flesch_kincaid_grade': 'mean',
    'latency_seconds': 'mean'
})

print(model_stats)

# Prompt-level stats (averaged across ALL models)
prompt_stats = df_success.groupby('prompt_name').agg({
    'format_pass': 'mean',
    'bertscore_f1': 'mean',
    'flesch_kincaid_grade': 'mean'
})

print(prompt_stats)

                            format_pass  bertscore_f1  flesch_kincaid_grade  \
model_name                                                                    
claude-3-5-sonnet-20240620     0.297980      0.859115              6.549417   
claude-3-opus-20240229         0.350962      0.859282              8.631613   
gemini-2.5-flash               0.490385      0.869389              5.414282   
gemini-2.5-pro                 0.418269      0.867278              5.904126   
gpt-3.5-turbo                  0.500000      0.872966              8.573203   
gpt-4-turbo                    0.500000      0.875456              8.852759   
gpt-4o                         0.500000      0.870473              7.313176   

                            latency_seconds  
model_name                                   
claude-3-5-sonnet-20240620         5.317576  
claude-3-opus-20240229             7.548510  
gemini-2.5-flash                  20.223798  
gemini-2.5-pro                    19.306779  
gpt-3.5-turb

In [11]:
# Check sample sizes per group
print("SAMPLE SIZES BY PROMPT")
print(df_success.groupby('prompt_name')['format_pass'].agg(['mean', 'count', 'sum']))

print("SAMPLE SIZES BY MODEL")
print(df_success.groupby('model_name')['format_pass'].agg(['mean', 'count', 'sum']))

print("FULL MODEL×PROMPT BREAKDOWN")
combo_detail = df_success.groupby(['model_name', 'prompt_name'])['format_pass'].agg(['mean', 'count', 'sum'])
print(combo_detail.sort_values('mean', ascending=False))

SAMPLE SIZES BY PROMPT
                        mean  count  sum
prompt_name                             
Default             0.900000    360  324
Engaging            0.000000    362    0
Explain like I'm 5  0.000000    362    0
With example        0.853591    362  309
SAMPLE SIZES BY MODEL
                                mean  count  sum
model_name                                      
claude-3-5-sonnet-20240620  0.297980    198   59
claude-3-opus-20240229      0.350962    208   73
gemini-2.5-flash            0.490385    208  102
gemini-2.5-pro              0.418269    208   87
gpt-3.5-turbo               0.500000    208  104
gpt-4-turbo                 0.500000    208  104
gpt-4o                      0.500000    208  104
FULL MODEL×PROMPT BREAKDOWN
                                                   mean  count  sum
model_name                 prompt_name                             
gemini-2.5-flash           With example        1.000000     52   52
gpt-3.5-turbo              With exam

In [9]:
# Best model×prompt combination
best_combo = df_success.groupby(['model_name', 'prompt_name']).agg({
    'format_pass': 'mean'
}).sort_values('format_pass', ascending=False).head(1)

print(best_combo)

                               format_pass
model_name       prompt_name              
gemini-2.5-flash With example          1.0


In [14]:
print("Overall model averages (across all 4 prompts):")
model_overall = df_success.groupby('model_name').agg({
    'format_pass': 'mean',
    'bertscore_f1': 'mean',
    'flesch_kincaid_grade': 'mean',
    'latency_seconds': 'mean'
}).round(3)
model_overall.columns = ['Format Compliance', 'BERTScore F1', 'Readability (Grade)', 'Latency (seconds)']
display(model_overall)

print("MODEL PERFORMANCE ON STRUCTURED PROMPTS ONLY (Default + With example):")
df_structured = df_success[df_success['prompt_name'].isin(['Default', 'With example'])]

print("Structured prompt performance by model:")
structured_by_model = df_structured.groupby('model_name').agg({
    'format_pass': ['mean', 'sum', 'count'],
    'bertscore_f1': 'mean',
    'flesch_kincaid_grade': 'mean',
    'latency_seconds': 'mean'
}).round(3)
structured_by_model.columns = ['Format Compliance', 'Passed Count', 'Total Cases', 
                                'BERTScore F1', 'Readability (Grade)', 'Latency (seconds)']
display(structured_by_model)

print("DETAILED MODEL×PROMPT BREAKDOWN (structured prompts only):")
structured_detail = df_structured.groupby(['model_name', 'prompt_name']).agg({
    'format_pass': ['mean', 'sum', 'count'],
    'bertscore_f1': 'mean',
    'flesch_kincaid_grade': 'mean',
    'latency_seconds': 'mean'
}).round(3)
structured_detail.columns = ['Format Compliance', 'Passed', 'Total', 
                              'BERTScore F1', 'Readability', 'Latency']
display(structured_detail.sort_values('Format Compliance', ascending=False))

print("PROMPT ENGINEERING IMPACT")
print("Prompt averages (across all 7 models):")
prompt_overall = df_success.groupby('prompt_name').agg({
    'format_pass': ['mean', 'sum', 'count'],
    'bertscore_f1': 'mean',
    'flesch_kincaid_grade': 'mean'
}).round(3)
prompt_overall.columns = ['Format Compliance', 'Passed Count', 'Total Cases', 
                          'BERTScore F1', 'Readability (Grade)']
display(prompt_overall)

print("STRUCTURED vs UNSTRUCTURED PROMPT COMPARISON:")
df_success['prompt_type'] = df_success['prompt_name'].apply(
    lambda x: 'Structured' if x in ['Default', 'With example'] else 'Unstructured'
)
prompt_type_comparison = df_success.groupby('prompt_type').agg({
    'format_pass': ['mean', 'sum', 'count'],
    'bertscore_f1': 'mean',
    'flesch_kincaid_grade': 'mean'
}).round(3)
prompt_type_comparison.columns = ['Format Compliance', 'Passed Count', 'Total Cases',
                                   'BERTScore F1', 'Readability (Grade)']
display(prompt_type_comparison)

print("TOP 10 MODEL×PROMPT COMBINATIONS (by format compliance):")
top_combos = df_success.groupby(['model_name', 'prompt_name']).agg({
    'format_pass': ['mean', 'sum', 'count'],
    'bertscore_f1': 'mean',
    'flesch_kincaid_grade': 'mean',
    'latency_seconds': 'mean'
}).round(3)
top_combos.columns = ['Format Compliance', 'Passed', 'Total', 
                      'BERTScore F1', 'Readability', 'Latency']
display(top_combos.sort_values('Format Compliance', ascending=False).head(10))

print("BOTTOM 5 MODEL×PROMPT COMBINATIONS (structured prompts only):")
bottom_structured = structured_detail.sort_values('Format Compliance', ascending=True).head(5)
display(bottom_structured)

print("CLAUDE 3.5 SONNET: DEFAULT vs WITH EXAMPLE COMPARISON:")
claude_comparison = df_structured[df_structured['model_name'] == 'claude-3-5-sonnet-20240620'].groupby('prompt_name').agg({
    'format_pass': ['mean', 'sum', 'count'],
    'bertscore_f1': 'mean',
    'flesch_kincaid_grade': 'mean',
    'latency_seconds': 'mean'
}).round(3)
claude_comparison.columns = ['Format Compliance', 'Passed', 'Total',
                             'BERTScore F1', 'Readability', 'Latency']
display(claude_comparison)

Overall model averages (across all 4 prompts):


,Format Compliance,BERTScore F1,Readability (Grade),Latency (seconds)
model_name,,,,
claude-3-5-sonnet-20240620,0.298,0.859,6.549,5.318
claude-3-opus-20240229,0.351,0.859,8.632,7.549
gemini-2.5-flash,0.490,0.869,5.414,20.224
gemini-2.5-pro,0.418,0.867,5.904,19.307
gpt-3.5-turbo,0.500,0.873,8.573,2.279
gpt-4-turbo,0.500,0.875,8.853,6.067
gpt-4o,0.500,0.870,7.313,4.192


MODEL PERFORMANCE ON STRUCTURED PROMPTS ONLY (Default + With example):
Structured prompt performance by model:


,Format Compliance,Passed Count,Total Cases,BERTScore F1,Readability (Grade),Latency (seconds)
model_name,,,,,,
claude-3-5-sonnet-20240620,0.602,59,98,0.847,8.281,5.207
claude-3-opus-20240229,0.702,73,104,0.851,9.495,8.838
gemini-2.5-flash,0.981,102,104,0.855,6.300,22.705
gemini-2.5-pro,0.837,87,104,0.861,7.044,19.522
gpt-3.5-turbo,1.000,104,104,0.850,8.478,2.718
gpt-4-turbo,1.000,104,104,0.855,9.672,7.456
gpt-4o,1.000,104,104,0.852,8.384,4.793


DETAILED MODEL×PROMPT BREAKDOWN (structured prompts only):


...

Format Compliance  Passed  Total  \
model_name                 prompt_name                                      
gemini-2.5-flash           With example              1.000      52     52   
gpt-3.5-turbo              With example              1.000      52     52   
                           Default                   1.000      52     52   
gemini-2.5-pro             With example              1.000      52     52   
gpt-4-turbo                Default                   1.000      52     52   
                           With example              1.000      52     52   
gpt-4o                     With example              1.000      52     52   
                           Default                   1.000      52     52   
gemini-2.5-flash           Default                   0.962      50     52   
claude-3-5-sonnet-20240620 Default                   0.938      45     48   

                                         BERTScore F1  Readability  Latency  
model_name                 prompt_name                                       
gemini-2.5-flash           With example         0.846        5.221   22.364  
gpt-3.5-turbo              With example         0.847        7.867    2.518  
                           Default              0.853        9.089    2.917  
gemini-2.5-pro             With example         0.853        6.146   20.442  
gpt-4-turbo                Default              0.859       10.993    8.416  
                           With example         0.850        8.351    6.496  
gpt-4o                     With example         0.849        7.285    4.693  
                           Default              0.856        9.483    4.892  
gemini-2.5-flash           Default              0.865        7.379   23.045  
claude-3-5-sonnet-20240620 Default              0.854        8.230    5.018

BOTTOM 5 MODEL×PROMPT COMBINATIONS (structured prompts only):


,,Format Compliance,Passed,Total,BERTScore F1,Readability,Latency
model_name,prompt_name,,,,,,
claude-3-5-sonnet-20240620,With example,0.280,14,50,0.840,8.330,5.389
claude-3-opus-20240229,With example,0.673,35,52,0.844,7.782,8.094
gemini-2.5-pro,Default,0.673,35,52,0.868,7.941,18.603
claude-3-opus-20240229,Default,0.731,38,52,0.858,11.208,9.582
claude-3-5-sonnet-20240620,Default,0.938,45,48,0.854,8.230,5.018


CLAUDE 3.5 SONNET: DEFAULT vs WITH EXAMPLE COMPARISON:


,Format Compliance,Passed,Total,BERTScore F1,Readability,Latency
prompt_name,,,,,,
Default,0.938,45,48,0.854,8.23,5.018
With example,0.280,14,50,0.840,8.33,5.389
